# Coursera Capstone Project

## The Battle of Neighborhoods (Week 1)


### Coursera Capstone - REPORT CONTENT

1.	Introduction Section : 
	⁃	Discussion of the business problem and the interested audience in this project.
2.	Data Section:  
	⁃	 Description of the data that will be used to solve the problem and the sources.
3.	Methodology section 
	⁃	Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4.	Results section 
	⁃	Discussion of the results.
5.	Discussion section 
	⁃	Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6.	Conclusion section 
	⁃	Report Conclusion.

In [20]:
# A description of the data and how it will be used to solve the problem. (15 marks)

# 2. Data Section:¶
### Description of the data and its sources that will be used to solve the problem

### Description of the Data:¶

The following data is required to  answer the issues of the problem:

- List of Venu Name and Catagory of Dammam, Saudi Arabia with their geodata (latitud and longitud) 
- List of Indian Resturant in Dammam with their address location 
- number of and distance to Indian restaurants in the neighborhood, if any
- distance of neighborhood from city center


### How the data will be used to solve the problem

The data will be used as follows:
- Use Foursquare and geopy data to map top 100 venues for all Dammam neighborhoods and clustered in groups
- Use foursquare and geopy data to map the location of Indian Resturant , separately and on top of the above clustered map in order to be able to identify the venues and catagory near city center
- centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
- coordinate of Dammam center will be obtained using Google Maps API geocoding of well known Dammam location 


### Reference of Venues Name & Catagory in Dammam

In [6]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [7]:

# Dammam, Saudi Arabia
address = 'Dammam, Saudi Arabia'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dammam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dammam are 26.4367824, 50.1039991.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [8]:
neighborhood_latitude=26.4367824
neighborhood_longitude=50.1039991

In [9]:
# @hidden cell
CLIENT_ID = 'KOHIW1QT3ENDODRSQ1U0VJ10A51EYY1F4PPD402FEB1GWXXJ' # Foursquare ID
CLIENT_SECRET = 'JCERQAIDIP0G3H2W1JA0VAUEKHJPM5WVZSVXW50XNTQTXTML' # Foursquare Secret
VERSION = '20200313'

In [36]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KOHIW1QT3ENDODRSQ1U0VJ10A51EYY1F4PPD402FEB1GWXXJ&client_secret=JCERQAIDIP0G3H2W1JA0VAUEKHJPM5WVZSVXW50XNTQTXTML&v=20200313&ll=26.4367824,50.1039991&radius=1500&limit=200'

In [37]:
results = requests.get(url).json()
#results

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head()

,name,categories,lat,lng
0,Golden Ideas,Gift Shop,26.437979,50.103441
1,شارع الجملة للمكتبات,Arts & Crafts Store,26.437353,50.102311
2,Red Dragon,Asian Restaurant,26.438572,50.106188
3,Nehari House Restaurant,Indian Restaurant,26.437293,50.106685
4,Horizon restaurant (مطعم الأفق),Indian Restaurant,26.438920,50.105348


In [40]:
SGnearby_venues.tail()

,name,categories,lat,lng
95,الخشي للدعاية والاعلان,Market,26.426012,50.096673
96,Shahnaz Iranian Restaurant | مطعم شاهيناز,Middle Eastern Restaurant,26.438352,50.118907
97,Dammam Shop,Cosmetics Shop,26.439662,50.104665
98,Dammam Palace Hotel,Hotel,26.428787,50.103735
99,Gulf Flower Hotel,Hotel,26.435730,50.107572


## Map of Dammam with venues Name

In [50]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=16)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        number_of_sides=4,
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

In [29]:
# Thanks !!